In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn import model_selection, preprocessing, linear_model
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import numpy as np

C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [3]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
gpu = gpus[0]
tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping 

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, MaxPooling1D, SimpleRNN, LSTM, GRU, Masking

In [4]:
df_train = pd.read_csv("D:/GeekBrains/nlp/les08/data/train.csv")
df_test = pd.read_csv("D:/GeekBrains/nlp/les08/data/test.csv")
df_val = pd.read_csv("D:/GeekBrains/nlp/les08/data/val.csv")

In [5]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [6]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation.replace('@', ''))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()

    txt = re.sub("@\S+ ", ' ', txt) #remove nicknames
#     txt = re.sub("RT ", ' ', txt)
    txt = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•`'·&()]|[+=]|[[]|[]]|[/]|", '', txt)
    txt = re.sub("[+=]|[[]|[]]|[/]|", '', txt)
    txt = re.sub("\\'", "'", txt)
    txt = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', txt)
    txt = re.sub(r'[\xad]', '', txt.strip())

    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [7]:
df_train['text'].iloc[1]

'RT @GalyginVadim: Ребята и девчата!\nВсе в кино!!! "Вот Это Любовь!"\nСегодня! Завтра!  И потом!)))))\n#вотэтолюбовь'

In [8]:
preprocess_text(df_train['text'].iloc[1])

C:\Users\snetkova\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Possible nested set at position 41
  if sys.path[0] == '':
C:\Users\snetkova\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Possible nested set at position 6
  del sys.path[0]


'rt ребята девчата кино любовь завтра вотэтолюбовь'

In [9]:
df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [10]:
text_corpus_train = df_train['text'].values
text_corpus_val = df_val['text'].values
text_corpus_test = df_test['text'].values

In [11]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_val)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_val = pad_sequences(sequences_val, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

In [12]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [13]:
res = {}

#### CNN

In [14]:
model1 = Sequential()
model1.add(Embedding(input_dim=word_count, output_dim=30, input_length=training_length))
model1.add(Conv1D(30, 3))
model1.add(Activation("relu"))
model1.add(GlobalMaxPool1D())
model1.add(Dense(10))
model1.add(Activation("relu"))
model1.add(Dense(1))
model1.add(Activation('sigmoid'))
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 24, 30)            5066010   
_________________________________________________________________
conv1d (Conv1D)              (None, 22, 30)            2730      
_________________________________________________________________
activation (Activation)      (None, 22, 30)            0         
_________________________________________________________________
global_max_pooling1d (Global (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                310       
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [15]:
model1.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
early_stopping=EarlyStopping(monitor='val_loss')
model1.fit(X_train, y_train, validation_data = (X_val,y_val),
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    callbacks=[early_stopping]
         )

Epoch 1/10
355/355 [==============================] - 23s 57ms/step - loss: 0.5827 - accuracy: 0.6890 - val_loss: 0.5350 - val_accuracy: 0.7243


In [17]:
res['cnn'] = model1.evaluate(X_val, y_val, batch_size=512, verbose=1)[1]
res

45/45 [==============================] - 0s 3ms/step - loss: 0.5350 - accuracy: 0.7243


{'cnn': 0.7243310213088989}

#### Simple RNN

In [18]:
model2 = Sequential()

model2.add(Embedding(input_dim=word_count, output_dim=30, input_length=training_length))
model2.add(SimpleRNN(64))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 24, 30)            5066010   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 64)                6080      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 5,076,315
Trainable params: 5,076,315
Non-trainable params: 0
_________________________________________________________________


In [19]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
early_stopping=EarlyStopping(monitor='val_loss')
model2.fit(X_train, y_train, validation_data = (X_val,y_val),
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    callbacks=[early_stopping]
         )

Epoch 1/10
355/355 [==============================] - 35s 98ms/step - loss: 0.5649 - accuracy: 0.6961 - val_loss: 0.5229 - val_accuracy: 0.7313


In [21]:
res['srnn'] = model2.evaluate(X_val, y_val, batch_size=512, verbose=1)[1]
res

45/45 [==============================] - 0s 7ms/step - loss: 0.5229 - accuracy: 0.7313


{'cnn': 0.7243310213088989, 'srnn': 0.7313406467437744}

#### LSTM

In [22]:
model3 = Sequential()

model3.add(Embedding(input_dim=word_count, output_dim=30, input_length=training_length))
# recurrent_dropout = 0.2 WARNING:tensorflow:Layer will not use cuDNN kernels since it doesn't meet the criteria.
model3.add(LSTM(64, recurrent_dropout=0.0))
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(1, activation='sigmoid'))
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 24, 30)            5066010   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24320     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 5,094,555
Trainable params: 5,094,555
Non-trainable params: 0
_________________________________________________________________


In [23]:
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
early_stopping=EarlyStopping(monitor='val_loss')
model3.fit(X_train, y_train, validation_data = (X_val,y_val),
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    callbacks=[early_stopping]
         )

Epoch 1/10
355/355 [==============================] - 24s 61ms/step - loss: 0.5639 - accuracy: 0.6993 - val_loss: 0.5321 - val_accuracy: 0.7265


In [25]:
res['lstm'] = model3.evaluate(X_val, y_val, batch_size=512, verbose=1)[1]
res

45/45 [==============================] - 0s 4ms/step - loss: 0.5321 - accuracy: 0.7265


{'cnn': 0.7243310213088989,
 'srnn': 0.7313406467437744,
 'lstm': 0.7265353202819824}

#### GRU

In [26]:
model4 = Sequential()

model4.add(Embedding(input_dim=word_count, output_dim=30, input_length=training_length))
# recurrent_dropout = 0.2 WARNING:tensorflow:Layer will not use cuDNN kernels since it doesn't meet the criteria.
model4.add(GRU(64, recurrent_dropout=0.0))
model4.add(Dense(64, activation='relu'))
model4.add(Dropout(0.5))
model4.add(Dense(1, activation='sigmoid'))
model4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 24, 30)            5066010   
_________________________________________________________________
gru (GRU)                    (None, 64)                18432     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 5,088,667
Trainable params: 5,088,667
Non-trainable params: 0
_________________________________________________________________


In [27]:
model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [28]:
early_stopping=EarlyStopping(monitor='val_loss')
model4.fit(X_train, y_train, validation_data = (X_val,y_val),
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    callbacks=[early_stopping]
         )

Epoch 1/10
355/355 [==============================] - 22s 59ms/step - loss: 0.5598 - accuracy: 0.7034 - val_loss: 0.5246 - val_accuracy: 0.7307


In [29]:
res['gru'] = model4.evaluate(X_val, y_val, batch_size=512, verbose=1)[1]
res

45/45 [==============================] - 0s 4ms/step - loss: 0.5246 - accuracy: 0.7307


{'cnn': 0.7243310213088989,
 'srnn': 0.7313406467437744,
 'lstm': 0.7265353202819824,
 'gru': 0.7306793928146362}

#### CNN + LSTM

In [30]:
model5 = Sequential()

model5.add(Embedding(input_dim=word_count, output_dim=30, input_length=training_length))
model5.add(Conv1D(30, 3, padding='same', activation='relu'))
model5.add(MaxPooling1D(pool_size=2))
# recurrent_dropout = 0.2 WARNING:tensorflow:Layer will not use cuDNN kernels since it doesn't meet the criteria.
model5.add(LSTM(64, recurrent_dropout=0.0))
model5.add(Dense(64, activation='relu'))
model5.add(Dropout(0.5))
model5.add(Dense(1, activation='sigmoid'))
model5.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 24, 30)            5066010   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 24, 30)            2730      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 12, 30)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24320     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                

In [31]:
model5.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
early_stopping=EarlyStopping(monitor='val_loss')
model5.fit(X_train, y_train, validation_data = (X_val,y_val),
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    callbacks=[early_stopping]
         )

Epoch 1/10
355/355 [==============================] - 23s 60ms/step - loss: 0.5684 - accuracy: 0.6923 - val_loss: 0.5196 - val_accuracy: 0.7318


In [33]:
res['cnn_lstm'] = model5.evaluate(X_val, y_val, batch_size=512, verbose=1)[1]
res

45/45 [==============================] - 0s 4ms/step - loss: 0.5196 - accuracy: 0.7318


{'cnn': 0.7243310213088989,
 'srnn': 0.7313406467437744,
 'lstm': 0.7265353202819824,
 'gru': 0.7306793928146362,
 'cnn_lstm': 0.731825590133667}

#### LSTM + CNN

In [34]:
model6 = Sequential()

model6.add(Embedding(input_dim=word_count, output_dim=30, input_length=training_length))
# recurrent_dropout = 0.2 WARNING:tensorflow:Layer will not use cuDNN kernels since it doesn't meet the criteria.
model6.add(LSTM(64, recurrent_dropout=0.0, return_sequences = True))
model6.add(Conv1D(64, 3, padding='same', activation='relu'))
model6.add(MaxPooling1D(pool_size=2))

model6.add(Dense(64, activation='relu'))
model6.add(Dropout(0.5))
model6.add(Dense(1, activation='sigmoid'))
model6.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 24, 30)            5066010   
_________________________________________________________________
lstm_2 (LSTM)                (None, 24, 64)            24320     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 24, 64)            12352     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 12, 64)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 12, 64)            4160      
_________________________________________________________________
dropout_4 (Dropout)          (None, 12, 64)            0         
_________________________________________________________________
dense_11 (Dense)             (None, 12, 1)            

In [35]:
model6.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
early_stopping=EarlyStopping(monitor='val_loss')
model6.fit(X_train, y_train, validation_data = (X_val,y_val),
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    callbacks=[early_stopping]
         )

Epoch 1/10
355/355 [==============================] - 23s 61ms/step - loss: 0.6697 - accuracy: 0.5474 - val_loss: 0.6604 - val_accuracy: 0.5580


In [37]:
res['lstm_cnn'] = model6.evaluate(X_val, y_val, batch_size=512, verbose=1)[1]
res

45/45 [==============================] - 0s 6ms/step - loss: 0.6604 - accuracy: 0.5580


{'cnn': 0.7243310213088989,
 'srnn': 0.7313406467437744,
 'lstm': 0.7265353202819824,
 'gru': 0.7306793928146362,
 'cnn_lstm': 0.731825590133667,
 'lstm_cnn': 0.5580170154571533}

In [38]:
pd.DataFrame.from_dict(res, orient='index', columns = ['accuracy']).sort_values(by = 'accuracy', ascending = False)

,accuracy
cnn_lstm,0.731826
srnn,0.731341
gru,0.730679
lstm,0.726535
cnn,0.724331
lstm_cnn,0.558017


Самая высокая точность получилась на совместной архитектуре CNN -> RNN